TABLEAU_PROJECT

In [1]:
import pandas as pd 
import pymongo
MongoClient = pymongo.MongoClient

import time

client = MongoClient()
db = client.companies

In [2]:
client.companies

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [3]:
cursor = db.companies.find({"$and": [ {"category_code":"software"} , {"founded_year" : {"$gte": 2000} }, {"offices.latitude" : {"$ne": None} }, {"offices.longitude" : {"$ne": None} } ] },{"name": 1, "company_code": 1,"offices.latitude":1, "offices.longitude":1})
ls_cursor= list(cursor)

#print(ls_cursor)

a = []
counter = 0

for i in ls_cursor:
    i = counter
    a.append(ls_cursor[i])
    counter += 1

In [4]:
df = pd.DataFrame(a)
df.head()

,_id,name,offices
0,52cdef7c4bab8bd675297d8c,Zoho,"[{'latitude': 37.692934, 'longitude': -121.904..."
1,52cdef7c4bab8bd675297e30,GoingOn,"[{'latitude': 37.782263, 'longitude': -122.392..."
2,52cdef7c4bab8bd675297e3e,spigit,"[{'latitude': 37.6637279, 'longitude': -121.87..."
3,52cdef7c4bab8bd675297e3c,Bazaarvoice,"[{'latitude': 30.407545, 'longitude': -97.7176..."
4,52cdef7c4bab8bd675297e57,GoLightly,"[{'latitude': 37.898575, 'longitude': -122.537..."


In [5]:
b = []
counter = 0

for i in df["offices"]:
    i = counter
    b.append(df["offices"][i])
    counter += 1

df_b = pd.DataFrame(b)
df_b = df_b[0] # nos quedamos sólo con la primera sucursal
df_b = pd.Series.to_frame(df_b) # convertimos a DF
df_b.head()

,0
0,"{'latitude': 37.692934, 'longitude': -121.904945}"
1,"{'latitude': 37.782263, 'longitude': -122.392142}"
2,"{'latitude': 37.6637279, 'longitude': -121.873..."
3,"{'latitude': 30.407545, 'longitude': -97.717667}"
4,"{'latitude': 37.898575, 'longitude': -122.537302}"


In [6]:
#print(df_b[0][65])
#print(df['coords'][65])
#len(df_b)
# df['coords'] = df_b[0]
# df.index -> RangeIndex(start=0, stop=1133, step=1)
#df_b.index

In [7]:
df_b = df_b.dropna() # eliminamos valores nulos
df_b.head()
df_b.index


Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1118, 1119, 1121, 1123, 1124, 1128, 1129, 1130, 1131, 1132],
           dtype='int64', length=958)

ls = list(range(len(df)))

for i in ls:
    if df['offices'][i][0].items() == None:
        df = df.drop(df.index[i], inplace=True)
len(df) 

In [8]:
df_b = df_b.reset_index(drop=False)
df_b.head()

,index,0
0,0,"{'latitude': 37.692934, 'longitude': -121.904945}"
1,1,"{'latitude': 37.782263, 'longitude': -122.392142}"
2,2,"{'latitude': 37.6637279, 'longitude': -121.873..."
3,3,"{'latitude': 30.407545, 'longitude': -97.717667}"
4,4,"{'latitude': 37.898575, 'longitude': -122.537302}"


In [9]:
df_b[0][65]['longitude']

-122.391843

In [10]:
longitude = []
latitude = []
counter = 0

for i in df_b[0]:
    i = counter
    longitude.append(df_b[0][i]['longitude'])
    latitude.append(df_b[0][i]['latitude'])
    counter += 1


In [11]:
df_b['lat'] = latitude
df_b['long'] = longitude

df_b.head()

,index,0,lat,long
0,0,"{'latitude': 37.692934, 'longitude': -121.904945}",37.692934,-121.904945
1,1,"{'latitude': 37.782263, 'longitude': -122.392142}",37.782263,-122.392142
2,2,"{'latitude': 37.6637279, 'longitude': -121.873...",37.663728,-121.873181
3,3,"{'latitude': 30.407545, 'longitude': -97.717667}",30.407545,-97.717667
4,4,"{'latitude': 37.898575, 'longitude': -122.537302}",37.898575,-122.537302


In [25]:
import geojson

{
    db.companies.find : {
       "$near": {
           "$geometry": { 
               type: "Point",  
               coordinates: [ 180, 90] 
            }
        },
    }
}

NameError: name 'coordinates' is not defined